### __睡眠段階予測コンペティション チュートリアル__

- 睡眠ポリグラフ（polysomnography: PSG）から睡眠の深さ（睡眠段階）を予測するタスク

- [コンペリンク](https://www.nishika.com/competitions/sleep/summary)

- [参考](https://colab.research.google.com/drive/1yA1Lexs9XxXqs34p79hDEPoOnHiOJMSU)

### __睡眠段階__

- 睡眠ポリグラフを解析し、**30秒を1epoch**として、epoch毎に睡眠段階を判定する

- 1epochに異なる睡眠段階が混在する場合は、最も多くの時間を占める睡眠段階を採用する

- 今回使用するデータセット [Sleep-EDF Database Expanded](https://www.physionet.org/content/sleep-edfx/1.0.0/) では睡眠段階は以下の8つ

    - Sleep stage W (覚醒状態)

    - Sleep stage R (レム睡眠)

        - 急速眼球運動

    - Sleep stage 1
    
    - Sleep stage 2
    
    - Sleep stage 3
    
    - Sleep stage 4

    - Sleep stage ? (不明)
    
    - Movement time

- 一方、米国睡眠医学会(AASM)によるガイドラインでは`Sleep stage 3`と`Sleep stage 4`を`Sleep stage 3/4`に統合している

- このコンペティションではAASMによる分類のラベル付けに変更する

- `Movement time`と`Sleep stage ?`は`sample_submission.csv`の正解データには含まれない

In [1]:
# AASMによる分類に変更
RANK_LABEL2ID = {
    'Movement time': -1,
    'Sleep stage ?': -1,
    'Sleep stage W': 0,
    'Sleep stage 1': 1,
    'Sleep stage 2': 2,
    'Sleep stage 3': 3,
    'Sleep stage 4': 3,
    'Sleep stage R': 4
}

# AASMによる分類
LABEL2ID = {
    'Movement time': -1,
    'Sleep stage ?': -1,
    'Sleep stage W': 0,
    'Sleep stage 1': 1,
    'Sleep stage 2': 2,
    'Sleep stage 3/4': 3,
    'Sleep stage R': 4
}

ID2LABEL = {v: k for k, v in LABEL2ID.items()}

### __Setup__

In [77]:
import datetime
from pathlib import Path
import pickle
from tqdm.notebook import tqdm

import pandas as pd
import numpy as np

from sklearn.model_selection import GroupKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

import mne

In [3]:
DATA_DIR = Path("../data")
EDF_DIR = DATA_DIR / "edf_data"
SUBMISSION_DIR = Path("../submission")

### __submissionの形式の確認__

In [4]:
sample_submission_df = pd.read_csv(SUBMISSION_DIR / "sample_submission.csv", parse_dates=[1])
sample_submission_df

,id,meas_time,condition
0,53c1555,1989-11-20 23:19:30,Sleep stage W
1,53c1555,1989-11-20 23:20:00,Sleep stage W
2,53c1555,1989-11-20 23:20:30,Sleep stage W
3,53c1555,1989-11-20 23:21:00,Sleep stage W
4,53c1555,1989-11-20 23:21:30,Sleep stage W
...,...,...,...
52291,9b444bb,1989-04-12 07:32:30,Sleep stage W
52292,9b444bb,1989-04-12 07:33:00,Sleep stage W
52293,9b444bb,1989-04-12 07:33:30,Sleep stage W
52294,9b444bb,1989-04-12 07:34:00,Sleep stage W


### __レコードデータ__

- 学習データ: `train_recoreds.csv`

- 訓練データ: `test_recoreds.csv`

__カラム説明__

* id: ID

* subject_id: 被験者のID

* night: 測定した夜のID

* sex: 性別

* psg: センサーデータのファイル名

* hypnogram: アノテーションデータのファイル名


__注意__  

- 一部の被験者はレコードデータ含まれてない

- そのため、データセットにあるデータをすべて訓練データとして利用してもよい

In [5]:
train = pd.read_csv(DATA_DIR / "train_records.csv")
test = pd.read_csv(DATA_DIR / "test_records.csv")

train.shape, test.shape

((108, 8), (45, 7))

### __EDFデータ__

- `*-PSG.edf`: __波形データ__

- `*-Hypnogram.edf`: __睡眠段階のデータ__

In [6]:
# パスを設定

train["hypnogram"] = train["hypnogram"].map(lambda x: str(EDF_DIR / x))
train["psg"] = train["psg"].map(lambda x: str(EDF_DIR / x))
test["psg"] = test["psg"].map(lambda x: str(EDF_DIR / x))

In [7]:
# 1行だけ選択
row = train.iloc[0]
row

id                                           3c1c5cf
subject_id                                   07c46da
night                                              1
age                                               90
sex                                             male
lights_off                                  23:28:00
psg                 ../data/edf_data/3c1c5cf-PSG.edf
hypnogram     ../data/edf_data/3c1c5cf-Hypnogram.edf
Name: 0, dtype: object

### __PSGデータ__

- ファイルの読み込み: `mne.io.read_raw_edf()`

    - `preload=False`でデータのメタ情報のみ読み込み、省メモリで済む

    - `.info`でメタデータを取得

    - `.to_data_frame()`でDataFrameに変換

- チャンネル詳細

    - EEG： 脳波。すべての予測段階の判定に必要であり、意識水準と対応して変化する

        - Fpz-Cz、Pz-Ozは頭に取り付けるセンサー箇所の箇所の電位差を表す
    
    - EOG： 眼電図。 眼球運動を測定。
    
        - horizontalは水平方向に取り付けた電位差を表す
    
    - EMG： オトガイ(顎下)筋電図。
    
    - Resp oro-nasal：口鼻呼吸
    
    - Temp rectal：直腸温 (原著に記載がないため、実際に直腸の温度を計測したかどうかは不明)
    
    - Event marker：各時刻で起きたイベントの時刻

In [8]:
# edfファイルの読み込み

psg_edf = mne.io.read_raw_edf(row["psg"], preload=False, verbose=False)
psg_edf

<RawEDF | 3c1c5cf-PSG.edf, 7 x 7890000 (78900.0 s), ~12 kB, data not loaded>

In [9]:
# メタ情報
# 辞書Like
psg_edf.info

<Info | 7 non-empty values
 bads: []
 ch_names: EEG Fpz-Cz, EEG Pz-Oz, EOG horizontal, Resp oro-nasal, EMG ...
 chs: 7 EEG
 custom_ref_applied: False
 highpass: 0.5 Hz
 lowpass: 100.0 Hz
 meas_date: 1989-11-13 16:35:00 UTC
 nchan: 7
 projs: []
 sfreq: 100.0 Hz
>

In [10]:
# チャンネル一覧
psg_edf.ch_names

['EEG Fpz-Cz',
 'EEG Pz-Oz',
 'EOG horizontal',
 'Resp oro-nasal',
 'EMG submental',
 'Temp rectal',
 'Event marker']

In [11]:
# DataFrameに変換
psg_df = psg_edf.to_data_frame()
psg_df

,time,EEG Fpz-Cz,EEG Pz-Oz,EOG horizontal,Resp oro-nasal,EMG submental,Temp rectal,Event marker
0,0.00,84.851770,-11.197558,-219.482051,8.800000e+07,3.234000,1.427420e+07,9.490000e+08
1,0.01,87.957998,13.197558,-215.531868,8.790847e+07,3.233153,1.428125e+07,9.499478e+08
2,0.02,96.862515,-3.908181,-209.112821,8.781710e+07,3.232296,1.428833e+07,9.509124e+08
3,0.03,106.181197,-2.644689,-204.175092,8.772580e+07,3.231429,1.429543e+07,9.518927e+08
4,0.04,102.557265,-25.095971,-198.249817,8.763449e+07,3.230552,1.430254e+07,9.528878e+08
...,...,...,...,...,...,...,...,...
7889995,78899.95,-0.776557,-3.422222,12.097436,6.878336e+07,3.304648,1.428788e+07,9.730920e+08
7889996,78899.96,2.950916,-1.284005,11.109890,6.845655e+07,3.304943,1.428799e+07,9.730803e+08
7889997,78899.97,-12.476679,-0.409280,-1.728205,6.811467e+07,3.305226,1.428810e+07,9.730652e+08
7889998,78899.98,-3.261538,-13.530159,6.172161,6.775786e+07,3.305496,1.428820e+07,9.730468e+08


In [12]:
# 時刻カラムの作成
meas_start = psg_edf.info["meas_date"]
meas_start = meas_start.replace(tzinfo=None)
psg_df["meas_time"] = pd.date_range(start=meas_start, periods=len(psg_df), freq=pd.Timedelta(1 / 100, unit="s")) # 100Hz
psg_df

,time,EEG Fpz-Cz,EEG Pz-Oz,EOG horizontal,Resp oro-nasal,EMG submental,Temp rectal,Event marker,meas_time
0,0.00,84.851770,-11.197558,-219.482051,8.800000e+07,3.234000,1.427420e+07,9.490000e+08,1989-11-13 16:35:00.000
1,0.01,87.957998,13.197558,-215.531868,8.790847e+07,3.233153,1.428125e+07,9.499478e+08,1989-11-13 16:35:00.010
2,0.02,96.862515,-3.908181,-209.112821,8.781710e+07,3.232296,1.428833e+07,9.509124e+08,1989-11-13 16:35:00.020
3,0.03,106.181197,-2.644689,-204.175092,8.772580e+07,3.231429,1.429543e+07,9.518927e+08,1989-11-13 16:35:00.030
4,0.04,102.557265,-25.095971,-198.249817,8.763449e+07,3.230552,1.430254e+07,9.528878e+08,1989-11-13 16:35:00.040
...,...,...,...,...,...,...,...,...,...
7889995,78899.95,-0.776557,-3.422222,12.097436,6.878336e+07,3.304648,1.428788e+07,9.730920e+08,1989-11-14 14:29:59.950
7889996,78899.96,2.950916,-1.284005,11.109890,6.845655e+07,3.304943,1.428799e+07,9.730803e+08,1989-11-14 14:29:59.960
7889997,78899.97,-12.476679,-0.409280,-1.728205,6.811467e+07,3.305226,1.428810e+07,9.730652e+08,1989-11-14 14:29:59.970
7889998,78899.98,-3.261538,-13.530159,6.172161,6.775786e+07,3.305496,1.428820e+07,9.730468e+08,1989-11-14 14:29:59.980


### __Hypnogramデータ__

- ファイルの読み込み: `mne.read_annotations()`

- カラム説明
    - onset：経過時間
    
        - UNIXエポック(1970年1月1日午前0時0分0秒からの経過秒数)を基準としているため、正確な日時ではない
    
    - duration：アノテーションの間隔時間
    
    - description：睡眠段階

- 最初と最後にかなり長い `Sleep Stage W` がある

In [13]:
annot = mne.read_annotations(row["hypnogram"])
annot_df = annot.to_data_frame()
annot_df

,onset,duration,description
0,1970-01-01 00:00:00,16290.0,Sleep stage W
1,1970-01-01 04:31:30,30.0,Sleep stage 1
2,1970-01-01 04:32:00,90.0,Sleep stage 2
3,1970-01-01 04:33:30,1890.0,Sleep stage W
4,1970-01-01 05:05:00,30.0,Sleep stage 1
...,...,...,...
132,1970-01-01 17:37:30,30.0,Sleep stage 1
133,1970-01-01 17:38:00,270.0,Sleep stage W
134,1970-01-01 17:42:30,30.0,Sleep stage 1
135,1970-01-01 17:43:00,15120.0,Sleep stage W


In [14]:
annot_df["description"].value_counts()

Sleep stage 1    47
Sleep stage 2    39
Sleep stage W    28
Sleep stage 3    12
Sleep stage R     8
Sleep stage ?     3
Name: description, dtype: int64

### __PSGデータとHypnogramデータの紐付け__

PSGデータを30秒毎に区切って1epochとして、`onset`と`duration`で時刻を紐付ける

また、前後にかなり長い`Sleep Stage W`があります。  

データ数を減らすため、適当に5時間で解析時間の切り捨てを行います

In [15]:
truncate_start_point = 3600 * 5 # 開始5時間後
truncate_end_point = (len(psg_edf)/100) - (3600 *5) # 終了5時間前

# 切り捨て
annot.crop(truncate_start_point, truncate_end_point, verbose=False)
annot.to_data_frame()

,onset,duration,description
0,1970-01-01 05:00:00,300.0,Sleep stage W
1,1970-01-01 05:05:00,30.0,Sleep stage 1
2,1970-01-01 05:05:30,180.0,Sleep stage 2
3,1970-01-01 05:08:30,17160.0,Sleep stage W
4,1970-01-01 09:54:30,30.0,Sleep stage 1
...,...,...,...
121,1970-01-01 16:44:00,270.0,Sleep stage 1
122,1970-01-01 16:48:30,150.0,Sleep stage W
123,1970-01-01 16:51:00,60.0,Sleep stage 1
124,1970-01-01 16:52:00,120.0,Sleep stage W


In [16]:
# annotationデータを紐づけ
psg_edf.set_annotations(annot, verbose=False) # emit_warning=False

<RawEDF | 3c1c5cf-PSG.edf, 7 x 7890000 (78900.0 s), ~12 kB, data not loaded>

30秒毎に分割された睡眠段階(__events__)を作成

- 何秒経過後に何が起こったのがわかる

- 1列目: 計測開始からの経過時間(10ms)

- 2列目: 意味無し(全て0)

- 3列目: アノテーションのID

In [17]:
# 30秒毎に分割された睡眠段階
events, _ = mne.events_from_annotations(
    psg_edf,
    event_id=RANK_LABEL2ID,
    chunk_duration=30., # 30秒ごとに区切る
    verbose=False,
)
events

array([[1800000,       0,       0],
       [1803000,       0,       0],
       [1806000,       0,       0],
       ...,
       [6081000,       0,       0],
       [6084000,       0,       1],
       [6087000,       0,       1]])

In [18]:
# 30秒毎に1epochとする
epoch = mne.Epochs(
    raw=psg_edf,
    events=events,
    event_id=LABEL2ID,
    tmin=0,
    tmax=30. - 1. / psg_edf.info['sfreq'],
    baseline=None,
    verbose=False,
    on_missing='ignore',
)

# メタデータの設定
epoch.info["temp"] = {
    "id": row["id"],
    "subject_id": row["subject_id"],
    "night": row["night"],
    "truncate_start_point": truncate_start_point,
    "truncate_end_point": truncate_end_point
}

epoch

Number of events,1430
Events,Movement time: 20Sleep stage 1: 122Sleep stage 2: 422Sleep stage 3/4: 27Sleep stage ?: 20Sleep stage R: 35Sleep stage W: 804
Time range,0.000 – 29.990 sec
Baseline,off


In [19]:
epoch_df = epoch.to_data_frame(verbose=False)
epoch_df

,time,condition,epoch,EEG Fpz-Cz,EEG Pz-Oz,EOG horizontal,Resp oro-nasal,EMG submental,Temp rectal,Event marker
0,0.00,Sleep stage W,0,-80.295971,0.368254,-42.711355,3.100000e+07,3.314000,1.427192e+07,9.530000e+08
1,0.01,Sleep stage W,0,-32.252991,7.074481,-30.367033,3.092531e+07,3.313483,1.427161e+07,9.526826e+08
2,0.02,Sleep stage W,0,-88.165079,6.782906,-54.561905,3.084926e+07,3.312963,1.427131e+07,9.523656e+08
3,0.03,Sleep stage W,0,15.893529,12.517216,-14.566300,3.077191e+07,3.312441,1.427100e+07,9.520494e+08
4,0.04,Sleep stage W,0,23.555556,15.724542,-29.873260,3.069332e+07,3.311918,1.427068e+07,9.517339e+08
...,...,...,...,...,...,...,...,...,...,...
4289995,29.95,Sleep stage 1,1429,7.092552,1.145788,3.209524,3.298500e+07,3.336563,1.424834e+07,9.379142e+08
4289996,29.96,Sleep stage 1,1429,0.155311,2.506471,7.653480,3.239297e+07,3.336464,1.424828e+07,9.377232e+08
4289997,29.97,Sleep stage 1,1429,6.574847,5.616606,2.715751,3.179811e+07,3.336358,1.424822e+07,9.375361e+08
4289998,29.98,Sleep stage 1,1429,-6.678388,9.990232,14.566300,3.120076e+07,3.336245,1.424818e+07,9.373531e+08


### __時間の設定__

epoch_dfのtimeカラムは0.00〜29.99秒の表示なので、それを時刻に変換する

これによりPSGデータとHypnogramデータの紐付けが完了する

In [20]:
epoch.info["meas_date"]

datetime.datetime(1989, 11, 13, 16, 35, tzinfo=datetime.timezone.utc)

In [21]:
# 切り捨てした分を追加して正しい開始時刻を計算
new_meas_date = epoch.info["meas_date"].replace(tzinfo=None) + datetime.timedelta(seconds=epoch.info["temp"]["truncate_start_point"])

# 時刻カラムの作成
epoch_df["meas_time"] = pd.date_range(start=new_meas_date, periods=len(epoch_df), freq=pd.Timedelta(1 / 100, unit="s"))
epoch_df

,time,condition,epoch,EEG Fpz-Cz,EEG Pz-Oz,EOG horizontal,Resp oro-nasal,EMG submental,Temp rectal,Event marker,meas_time
0,0.00,Sleep stage W,0,-80.295971,0.368254,-42.711355,3.100000e+07,3.314000,1.427192e+07,9.530000e+08,1989-11-13 21:35:00.000
1,0.01,Sleep stage W,0,-32.252991,7.074481,-30.367033,3.092531e+07,3.313483,1.427161e+07,9.526826e+08,1989-11-13 21:35:00.010
2,0.02,Sleep stage W,0,-88.165079,6.782906,-54.561905,3.084926e+07,3.312963,1.427131e+07,9.523656e+08,1989-11-13 21:35:00.020
3,0.03,Sleep stage W,0,15.893529,12.517216,-14.566300,3.077191e+07,3.312441,1.427100e+07,9.520494e+08,1989-11-13 21:35:00.030
4,0.04,Sleep stage W,0,23.555556,15.724542,-29.873260,3.069332e+07,3.311918,1.427068e+07,9.517339e+08,1989-11-13 21:35:00.040
...,...,...,...,...,...,...,...,...,...,...,...
4289995,29.95,Sleep stage 1,1429,7.092552,1.145788,3.209524,3.298500e+07,3.336563,1.424834e+07,9.379142e+08,1989-11-14 09:29:59.950
4289996,29.96,Sleep stage 1,1429,0.155311,2.506471,7.653480,3.239297e+07,3.336464,1.424828e+07,9.377232e+08,1989-11-14 09:29:59.960
4289997,29.97,Sleep stage 1,1429,6.574847,5.616606,2.715751,3.179811e+07,3.336358,1.424822e+07,9.375361e+08,1989-11-14 09:29:59.970
4289998,29.98,Sleep stage 1,1429,-6.678388,9.990232,14.566300,3.120076e+07,3.336245,1.424818e+07,9.373531e+08,1989-11-14 09:29:59.980


In [22]:
def epoch_to_df(epoch):
    df = epoch.to_data_frame(verbose=False)
    new_meas_date = epoch.info["meas_date"].replace(tzinfo=None) + datetime.timedelta(seconds=epoch.info["temp"]["truncate_start_point"])
    df["meas_time"] = pd.date_range(start=new_meas_date, periods=len(df), freq=pd.Timedelta(1 / 100, unit="s"))
    return df

epoch_to_df(epoch)

,time,condition,epoch,EEG Fpz-Cz,EEG Pz-Oz,EOG horizontal,Resp oro-nasal,EMG submental,Temp rectal,Event marker,meas_time
0,0.00,Sleep stage W,0,-80.295971,0.368254,-42.711355,3.100000e+07,3.314000,1.427192e+07,9.530000e+08,1989-11-13 21:35:00.000
1,0.01,Sleep stage W,0,-32.252991,7.074481,-30.367033,3.092531e+07,3.313483,1.427161e+07,9.526826e+08,1989-11-13 21:35:00.010
2,0.02,Sleep stage W,0,-88.165079,6.782906,-54.561905,3.084926e+07,3.312963,1.427131e+07,9.523656e+08,1989-11-13 21:35:00.020
3,0.03,Sleep stage W,0,15.893529,12.517216,-14.566300,3.077191e+07,3.312441,1.427100e+07,9.520494e+08,1989-11-13 21:35:00.030
4,0.04,Sleep stage W,0,23.555556,15.724542,-29.873260,3.069332e+07,3.311918,1.427068e+07,9.517339e+08,1989-11-13 21:35:00.040
...,...,...,...,...,...,...,...,...,...,...,...
4289995,29.95,Sleep stage 1,1429,7.092552,1.145788,3.209524,3.298500e+07,3.336563,1.424834e+07,9.379142e+08,1989-11-14 09:29:59.950
4289996,29.96,Sleep stage 1,1429,0.155311,2.506471,7.653480,3.239297e+07,3.336464,1.424828e+07,9.377232e+08,1989-11-14 09:29:59.960
4289997,29.97,Sleep stage 1,1429,6.574847,5.616606,2.715751,3.179811e+07,3.336358,1.424822e+07,9.375361e+08,1989-11-14 09:29:59.970
4289998,29.98,Sleep stage 1,1429,-6.678388,9.990232,14.566300,3.120076e+07,3.336245,1.424818e+07,9.373531e+08,1989-11-14 09:29:59.980


### __submission形式へ変換__

epoch毎にグループ化してtimeの最小値を取ることでepochの開始時間を取得し、submissionする形式に変換できる

In [23]:
def epoch_to_sub_df(epoch_df, id, is_train):
    cols = ["id", "meas_time"]
    
    # 訓練データにはアノテーション追加
    if is_train:
        cols.append("condition")
    
    label_df = epoch_df.loc[epoch_df.groupby("epoch")["time"].idxmin()].reset_index(drop=True)
    label_df["id"] = id
    
    return label_df[cols]

In [24]:
epoch_to_sub_df(epoch_df, epoch.info["temp"]["id"], is_train=True)

,id,meas_time,condition
0,3c1c5cf,1989-11-13 21:35:00,Sleep stage W
1,3c1c5cf,1989-11-13 21:35:30,Sleep stage W
2,3c1c5cf,1989-11-13 21:36:00,Sleep stage W
3,3c1c5cf,1989-11-13 21:36:30,Sleep stage W
4,3c1c5cf,1989-11-13 21:37:00,Sleep stage W
...,...,...,...
1425,3c1c5cf,1989-11-14 09:27:30,Sleep stage W
1426,3c1c5cf,1989-11-14 09:28:00,Sleep stage W
1427,3c1c5cf,1989-11-14 09:28:30,Sleep stage W
1428,3c1c5cf,1989-11-14 09:29:00,Sleep stage 1


### __テストデータの整形__

テストデータにはアノテーションファイルがないので、`events`の行列を一から作成する必要がある

評価の開始と終了時刻はsample submissionにあるので、前後の切り捨て時刻を計算する

アノテーションのラベルIDは`0`としてダミーで生成する

In [25]:
# 1行だけ選択
test_row = test.iloc[0]
psg_edf = mne.io.read_raw_edf(test_row["psg"], include=["EEG Fpz-Cz"], verbose=False)

# PSGの開始時刻
start_psg_date = psg_edf.info["meas_date"]
# 計算のためにtimezoneは消去
start_psg_date = start_psg_date.replace(tzinfo=None)

# sample submissionから評価される時間レンジを取得します
test_start_time = sample_submission_df[sample_submission_df["id"]==test_row["id"]]["meas_time"].min()
test_end_time = sample_submission_df[sample_submission_df["id"]==test_row["id"]]["meas_time"].max()
# psgの計測開始、評価の対象の開始、評価の対象の終了
print(f"psg start: {start_psg_date},  test start: {test_start_time}, test end: {test_end_time}")

# psgの計測時間から評価の対象の開始と終了を経過時間(秒)になおす
truncate_start_point = int((test_start_time - start_psg_date).total_seconds())
truncate_end_point = int((test_end_time - start_psg_date).total_seconds())+30
print(f"event start second: {truncate_start_point}, event end second: {truncate_end_point} ")

# 30秒毎にデータ点を生成
event_range = list(range(truncate_start_point, truncate_end_point, 30))
events = np.zeros((len(event_range), 3), dtype=int)
events[:, 0] = event_range

# 秒を10m秒に変換する
events = events * 100

events

psg start: 1989-11-20 15:16:00,  test start: 1989-11-20 23:19:30, test end: 1989-11-21 08:10:30
event start second: 29010, event end second: 60900 


array([[2901000,       0,       0],
       [2904000,       0,       0],
       [2907000,       0,       0],
       ...,
       [6081000,       0,       0],
       [6084000,       0,       0],
       [6087000,       0,       0]])

In [26]:
# Epochの作成
epoch = mne.Epochs(raw=psg_edf,
    events=events,
    event_id={'Sleep stage W': 0},
    tmin=0,
    tmax=30. - 1. / psg_edf.info['sfreq'],
    baseline=None,
    verbose=False
)

In [27]:
epoch.to_data_frame(verbose=False)

,time,condition,epoch,EEG Fpz-Cz
0,0.00,Sleep stage W,0,-10.738462
1,0.01,Sleep stage W,0,-13.220513
2,0.02,Sleep stage W,0,-31.481319
3,0.03,Sleep stage W,0,-10.915751
4,0.04,Sleep stage W,0,-39.902564
...,...,...,...,...
3188995,29.95,Sleep stage W,1062,-37.065934
3188996,29.96,Sleep stage W,1062,-34.761172
3188997,29.97,Sleep stage W,1062,-37.331868
3188998,29.98,Sleep stage W,1062,-22.882784


### __関数化__

In [28]:
# 以上のデータ整形を関数化する
def read_and_set_annotation(record_df, include=None, is_test=False):
    whole_epoch_data = []

    for row_id, row in tqdm(record_df.iterrows(), total=len(record_df)):        
        # PSGファイルとHypnogram(アノテーションファイルを読み込む)
        psg_edf = mne.io.read_raw_edf(row["psg"], include=include, verbose=False)
        
        if not is_test:
            # 訓練データの場合
            annot = mne.read_annotations(row["hypnogram"])

            # 切り捨て
            truncate_start_point = 3600 * 5
            truncate_end_point = (len(psg_edf)/100) - (3600 *5)
            annot.crop(truncate_start_point, truncate_end_point, verbose=False)

            # アノテーションデータの切り捨て
            psg_edf.set_annotations(annot, emit_warning=False)
            events, _ = mne.events_from_annotations(psg_edf, event_id=RANK_LABEL2ID, chunk_duration=30., verbose=False)
            
            event_id = LABEL2ID
        else:
            # テストデータの場合
            start_psg_date = psg_edf.info["meas_date"]
            start_psg_date = start_psg_date.replace(tzinfo=None)

            test_start_time = sample_submission_df[sample_submission_df["id"]==row["id"]]["meas_time"].min()
            test_end_time = sample_submission_df[sample_submission_df["id"]==row["id"]]["meas_time"].max()
            
            truncate_start_point = int((test_start_time - start_psg_date).total_seconds())
            truncate_end_point = int((test_end_time- start_psg_date).total_seconds())+30
            
            event_range = list(range(truncate_start_point, truncate_end_point, 30))
            events = np.zeros((len(event_range), 3), dtype=int)
            events[:, 0] = event_range
            events = events * 100
            
            event_id = {'Sleep stage W': 0}

        # 30秒毎に1epochとする
        tmax = 30. - 1. / psg_edf.info['sfreq']
        epoch = mne.Epochs(raw=psg_edf, events=events, event_id=event_id, tmin=0, tmax=tmax, baseline=None, verbose=False, on_missing='ignore')
        
        # 途中でデータが落ちてないかチェック
        assert len(epoch.events) * 30 == truncate_end_point - truncate_start_point
        
        # メタデータを追加
        epoch.info["temp"] = {
            "id": row["id"],
            "subject_id": row["subject_id"],
            "night": row["night"],
            "age": row["age"],
            "sex": row["sex"],
            "truncate_start_point": truncate_start_point
        }

        whole_epoch_data.append(epoch)

    return whole_epoch_data 

In [29]:
# Epochデータセットの作成
train_epochs = read_and_set_annotation(train, include=["EEG Fpz-Cz"], is_test=False)
test_epochs = read_and_set_annotation(test, include=["EEG Fpz-Cz"], is_test=True)

  0%|          | 0/108 [00:00<?, ?it/s]

  0%|          | 0/45 [00:00<?, ?it/s]

In [78]:
# pickleファイルに保存
with open(DATA_DIR / 'train_epochs.pickle', mode='wb') as f:
    pickle.dump(train_epochs, f)

with open(DATA_DIR / 'test_epochs.pickle', mode='wb') as f:
    pickle.dump(test_epochs, f)

### __特徴量エンジニアリング__
`MNE`ライブラリで紹介されている、パワースペクトル密度を計算

https://mne.tools/stable/auto_tutorials/clinical/60_sleep.html#feature-engineering

In [54]:
def eeg_power_band(epochs):
    # 周波数帯
    # FREQ_BANDS = {
    #     "delta": [0.5, 4.5],
    #     "theta": [4.5, 8.5],
    #     "alpha": [8.5, 11.5],
    #     "sigma": [11.5, 15.5],
    #     "beta": [15.5, 30]
    # }

    FREQ_BANDS = {str(i): [i+0.5, i+1.5] for i in range(29)}

    spectrum = epochs.compute_psd(picks='eeg', fmin=0.5, fmax=30. ,verbose=False)
    psds, freqs = spectrum.get_data(return_freqs=True)
    psds /= np.sum(psds, axis=-1, keepdims=True) # 正規化

    X = []
    for fmin, fmax in FREQ_BANDS.values():
        psds_band = psds[:, :, (freqs >= fmin) & (freqs < fmax)].mean(axis=-1)
        X.append(psds_band.reshape(len(psds), -1))

    return np.concatenate(X, axis=1)

In [55]:
train_df = []
for epoch in tqdm(train_epochs):
    # 波形をdataframe化
    epoch_df = epoch_to_df(epoch)
    # submit形式のデータフレーム生成
    sub_df = epoch_to_sub_df(epoch_df, epoch.info["temp"]["id"], is_train=True)
    
    # パワースペクトル密度計算
    feature_df = pd.DataFrame(eeg_power_band(epoch))
    
    _df = pd.concat([sub_df, feature_df], axis=1)
    # 必要ないラベルがある場合は除外する
    _df = _df[~_df["condition"].isin(["Sleep stage ?", "Movement time"])]
    
    train_df.append(_df)
    
train_df = pd.concat(train_df).reset_index(drop=True)

  0%|          | 0/108 [00:00<?, ?it/s]

In [56]:
train_df["condition"].value_counts()

Sleep stage W      74202
Sleep stage 2      47018
Sleep stage R      17995
Sleep stage 1      13878
Sleep stage 3/4     8517
Name: condition, dtype: int64

In [57]:
# ラベルIDに変換
train_df["condition"] = train_df["condition"].map(LABEL2ID)
train_df

,id,meas_time,condition,0,1,2,3,4,5,6,...,19,20,21,22,23,24,25,26,27,28
0,3c1c5cf,1989-11-13 21:35:00,0,0.004616,0.001328,0.000822,0.000610,0.000516,0.000747,0.000635,...,0.000916,0.001337,0.000968,0.000952,0.001642,0.001664,0.001171,0.001728,0.002675,0.001485
1,3c1c5cf,1989-11-13 21:35:30,0,0.003233,0.001903,0.000985,0.000809,0.000601,0.000410,0.000681,...,0.000675,0.000878,0.001105,0.001014,0.001750,0.001276,0.001902,0.001946,0.002010,0.002131
2,3c1c5cf,1989-11-13 21:36:00,0,0.004094,0.001164,0.001113,0.000768,0.000684,0.000439,0.000697,...,0.000719,0.000719,0.001054,0.001058,0.002011,0.001866,0.001927,0.001174,0.001672,0.002610
3,3c1c5cf,1989-11-13 21:36:30,0,0.005677,0.002688,0.001515,0.000870,0.000605,0.000557,0.000495,...,0.000723,0.001097,0.001267,0.001251,0.000773,0.001328,0.001029,0.001503,0.001450,0.001705
4,3c1c5cf,1989-11-13 21:37:00,0,0.002269,0.001916,0.001071,0.000781,0.000558,0.000824,0.000747,...,0.001000,0.000991,0.000954,0.001630,0.001672,0.001502,0.001535,0.002137,0.001912,0.001872
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161605,5edb9d9,1990-03-14 08:17:30,1,0.015107,0.005688,0.002157,0.001223,0.000971,0.000784,0.000654,...,0.000244,0.000180,0.000213,0.000354,0.000282,0.000178,0.000155,0.000110,0.000130,0.000063
161606,5edb9d9,1990-03-14 08:18:00,1,0.014826,0.003570,0.002878,0.001603,0.001102,0.000687,0.000414,...,0.000266,0.000176,0.000233,0.000215,0.000157,0.000171,0.000118,0.000097,0.000076,0.000069
161607,5edb9d9,1990-03-14 08:18:30,2,0.008396,0.006922,0.004313,0.002478,0.001329,0.001061,0.000764,...,0.000109,0.000151,0.000142,0.000168,0.000114,0.000094,0.000086,0.000085,0.000054,0.000054
161608,5edb9d9,1990-03-14 08:19:00,2,0.011295,0.004147,0.004587,0.002570,0.001284,0.000659,0.000550,...,0.000184,0.000211,0.000144,0.000120,0.000079,0.000083,0.000065,0.000058,0.000039,0.000040


In [58]:
test_df = []
for epoch in tqdm(test_epochs):
    # 波形をdataframe化
    epoch_df = epoch_to_df(epoch)
    # submit形式のデータフレーム生成
    sub_df = epoch_to_sub_df(epoch_df, epoch.info["temp"]["id"], is_train=False)

    # パワースペクトル密度計算
    feature_df = pd.DataFrame(eeg_power_band(epoch))
    
    _df = pd.concat([sub_df, feature_df], axis=1)
    
    test_df.append(pd.concat([sub_df, feature_df], axis=1))
    
test_df = pd.concat(test_df)

  0%|          | 0/45 [00:00<?, ?it/s]

### __学習__

train: 80％
valid: 20%

モデル: ランダムフォレスト

In [70]:
X = train_df.drop(columns=["id", "meas_time", "condition"])
y = train_df["condition"]

cv_scores = []
gkf = GroupKFold(n_splits=5)
for i, (train_index, valid_index) in tqdm(enumerate(gkf.split(X, y, train_df["id"])), total=5):
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    
    model = RandomForestClassifier(
        n_estimators=100,
        random_state=32,
        max_depth=40
    )
    model.fit(X_train, y_train)
    score = accuracy_score(y_valid, model.predict(X_valid))
    tqdm.write(f"Fold: {i+1} Accuracy: {score}")
    cv_scores.append(score)
print(f"CV Score {np.mean(cv_scores)}")

  0%|          | 0/5 [00:00<?, ?it/s]

Fold: 1 Accuracy: 0.760634065732297
Fold: 2 Accuracy: 0.7570924704799877
Fold: 3 Accuracy: 0.7314673348407994
Fold: 4 Accuracy: 0.7431310908519334
Fold: 5 Accuracy: 0.775471045264383
CV Score 0.7535592014338801


### __提出ファイルの作成__

In [73]:
sample_submission_df["condition"] = model.predict(test_df.drop(columns=["id", "meas_time"]))
sample_submission_df["condition"] = sample_submission_df["condition"].map(ID2LABEL)
sample_submission_df.to_csv(SUBMISSION_DIR / "baseline.csv", index=False)
sample_submission_df

,id,meas_time,condition
0,53c1555,1989-11-20 23:19:30,Sleep stage W
1,53c1555,1989-11-20 23:20:00,Sleep stage W
2,53c1555,1989-11-20 23:20:30,Sleep stage W
3,53c1555,1989-11-20 23:21:00,Sleep stage W
4,53c1555,1989-11-20 23:21:30,Sleep stage W
...,...,...,...
52291,9b444bb,1989-04-12 07:32:30,Sleep stage W
52292,9b444bb,1989-04-12 07:33:00,Sleep stage W
52293,9b444bb,1989-04-12 07:33:30,Sleep stage W
52294,9b444bb,1989-04-12 07:34:00,Sleep stage W


In [74]:
sample_submission_df["condition"].value_counts()

Sleep stage 2      21218
Sleep stage W      19972
Sleep stage R       5943
Sleep stage 3/4     2971
Sleep stage 1       2192
Name: condition, dtype: int64